# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [1]:
# Install Required Packages
# %pip install optiguide

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import autogen
import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

Note: Pyomo not loaded


In [3]:
config_list = [{'model': 'llama3.2',
  'api_type': 'openai',
  'api_key': 'api_key',
  'base_url': 'http://localhost:11434/v1'}]

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/what-if/benchmark/application/coffee.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



In [5]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
%%capture
agent = OptiGuideAgent(
    name="OptiGuideCoffeeExample",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create a user's question.

In [7]:
ans = user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")

user (to OptiGuideCoffeeExample):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuideCoffeeExample):

Based on the provided code, I will add a constraint that prohibits shipping from supplier 1 to roastery 2. Here is the modified code snippet:

```python
# OPTIGUIDE *** CODE GOES HERE
model.addConstr(x[('supplier1', 'roastery2')] == 0)
```

This line of code adds a new constraint that the flow through the ship from supplier 1 to roastery 2 should be equal to zero, effectively prohibiting this shipping route.

The updated source code with the added line would look like this:

```python
import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_sup

In [8]:
print(ans.summary)

Sorry. I cannot answer your question.


In [9]:
ans2 = user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")

user (to OptiGuideCoffeeExample):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to writer):


Answer Code:


--------------------------------------------------------------------------------
[autogen.oai.client: 02-16 09:22:55] {577} WARNING - Model llama3.2 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
writer (to OptiGuideCoffeeExample):

Based on the provided code, here's an example answer with codes:

I'd like to add a constraint that sets the total shipping cost. Let's say we want all the coffee beans shipped out by roasteries to cafes at a maximum of 2400 units per day.

```python
# OPTIGUIDE *** CODE GOES HERE

# Add total shipping cost constraint
max_shipping_cost = 2400
model.addConstr(
    sum(x[i] for i in shipping_cost_from

In [10]:
print(ans2.summary)

**Optimization Results Comparison**

We have re-run the optimization problem with our additional constraint, and here are the comparison results:

**Original Optimal Value:** 2470.0 (achieved with the initial model)

**New Optimal Value:** **2470.0** (no change observed in this case, but let's analyze further)

The new constraint we added sets a limit on the total shipping cost to be maximized at 2400 units per day. However, surprisingly, our initial objective value remains unchanged at 2470.0.

This suggests that the current supply and demand constraints are such that increasing the shipping capacity within the allowed limits does not reduce the overall cost. In other words, the extra constraint was too mild to make a difference in this particular scenario.

**Impact of Adjustment:**

- **No Change in Optimal Value**: The new objective value remains identical at 2470.0.
  
This outcome should come as no surprise given the constrained conditions and minimal reduction possible due to th